# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filelocation=os.path.join("..","output_data","cities.csv")
citiesdf=pd.read_csv(filelocation)
citiesdf=citiesdf[["City","Country","Lat","Lng","Temperature (F)","Humidity (%)","Wind Speed (mph)","Cloudiness (%)"]]
citiesdf

,City,Country,Lat,Lng,Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%)
0,okhotsk,RU,59.38,143.30,-7.38,71.0,9.08,0.0
1,camana,PE,-16.62,-72.71,68.16,87.0,5.93,97.0
2,mar del plata,AR,-38.00,-57.56,62.01,71.0,4.70,0.0
3,georgetown,MY,5.41,100.34,84.20,79.0,8.05,20.0
4,saint-philippe,RE,-21.36,55.77,69.80,68.0,9.17,26.0
...,...,...,...,...,...,...,...,...
555,safranbolu,TR,41.25,32.69,35.56,78.0,4.68,98.0
556,zhumadian,CN,32.98,114.03,44.46,53.0,5.91,0.0
557,aloleng,PH,16.13,119.78,82.96,54.0,5.82,66.0
558,zhigansk,RU,66.77,123.37,-30.12,87.0,8.12,6.0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
figure_layout = {
    'width': '720px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout,zoom_level=1.6,center=(0,0))
citycoords=citiesdf[["Lat","Lng"]]
humidities=citiesdf["Humidity (%)"].astype(float)
humidities

0      71.0
1      87.0
2      71.0
3      79.0
4      68.0
       ... 
555    78.0
556    53.0
557    54.0
558    87.0
559    81.0
Name: Humidity (%), Length: 560, dtype: float64

In [4]:
heat_layer = gmaps.heatmap_layer(citycoords, weights=humidities,max_intensity=humidities.max())

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
idealcitiesdf=citiesdf.loc[(citiesdf["Temperature (F)"]<80) & (citiesdf["Temperature (F)"]>60)&(citiesdf["Cloudiness (%)"]<20)&(citiesdf["Wind Speed (mph)"]<5)]
idealcitiesdf=idealcitiesdf.dropna()
idealcitiesdf

,City,Country,Lat,Lng,Temperature (F),Humidity (%),Wind Speed (mph),Cloudiness (%)
2,mar del plata,AR,-38.00,-57.56,62.01,71.0,4.70,0.0
65,ailigandi,PA,9.23,-78.02,70.72,95.0,2.33,13.0
67,chuy,UY,-33.70,-53.46,60.30,76.0,4.99,0.0
69,axim,GH,4.87,-2.24,75.96,91.0,1.48,3.0
124,chum phae,TH,16.54,102.10,76.51,56.0,2.77,11.0
131,benemerito de las americas,MX,16.52,-90.64,70.74,84.0,1.81,19.0
146,mumford,GH,5.26,-0.76,74.39,95.0,2.19,15.0
177,harper,LR,4.38,-7.72,76.64,90.0,1.52,16.0
202,coxim,BR,-18.51,-54.76,78.28,58.0,4.85,17.0
218,lazaro cardenas,MX,17.96,-102.20,72.97,79.0,2.62,0.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
url="https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
idealcitiesdf["Hotel Name"]=None
idealcitiesdf["Hotel Lat"]=None
idealcitiesdf["Hotel Lng"]=None
hotels=[]
params={
    "key":g_key,
    "radius":"5000",
    "type":"lodging"
}
for index,row in idealcitiesdf.iterrows():    
    params['location']=str(row['Lat'])+","+str(row['Lng'])    
    response=requests.get(url,params=params).json()
#     print(response.url)
#     response=response.json()
#     pprint(response.json())
    try:
        idealcitiesdf.loc[index,"Hotel Name"]=response["results"][0]["name"]
        location=(response["results"][0]["geometry"]["location"]["lat"],response["results"][0]["geometry"]["location"]["lng"])
        idealcitiesdf.loc[index,"Hotel Lat"]=response["results"][0]["geometry"]["location"]["lat"]
        idealcitiesdf.loc[index,"Hotel Lng"]=response["results"][0]["geometry"]["location"]["lng"]        
        hotels.append({"Hotel Name":response["results"][0]["name"],"City":row["City"],"location":location,"Country":"test"})
    except(IndexError):
        print(row["City"])
    
    

dicabisagan
bozoum


In [7]:
idealcitiesdroppeddf=idealcitiesdf.dropna()


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in idealcitiesdroppeddf.iterrows()]
locations = idealcitiesdroppeddf[["Hotel Lat", "Hotel Lng"]]

In [9]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…